In [1]:
! pip install torch torchvision
! pip install torchsummary

In [2]:
import torch
import torch.nn as nn
from torch import optim
import torchvision.models as models
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from torchvision import transforms, models
import matplotlib.pyplot as plt
from torch.utils.data import random_split

torch.manual_seed(17)

# HYPERPARAMS

In [3]:
DEBUG = True
NUM_INPUT_CHANNELS = 3
NUM_OUTPUT_CHANNELS = 1

NUM_EPOCHS = 20

LR = 1e-6
MOMENTUM = 0.9
BATCH_SIZE = 16